### Editando state

Breakpoints servem para parar a execução, mas também são uma oportunidade para modificar o state do grapho

In [ ]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a by b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine the control flow
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(interrupt_before=["assistant"], checkpointer=memory)

# Show
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Input
initial_input = {"messages": "Multiply 2 and 3"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

Aplicar um update do state

In [ ]:
graph.update_state(
    thread,
    {"messages": [HumanMessage(content="No, actually multiply 3 and 3 !")]}
)

Verificando cada mensagem no state

In [ ]:
new_state= graph.get_state(thread).values
for m in new_state["messages"]:
    m.pretty_print()

Prosseguindo com o agente

In [ ]:
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"].pretty_print()

Repetindo para finalizar, já que ele para sempre ue chega no assistant

In [ ]:
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"].pretty_print()

### Aguardando entrada do usuário
Portanto, fica claro que podemos editar o estado do nosso agente após um ponto de interrupção.

Agora, e se quisermos permitir que o feedback humano realize essa atualização de estado?

Adicionaremos um nó que servirá como um marcador para o feedback humano dentro do nosso agente.

Este nó `human_feedback` permite que o usuário adicione feedback diretamente ao estado.

Especificamos o ponto de interrupção usando `interrupt_before` em nosso nó `human_feedback`.

Configuramos um ponto de verificação para salvar o estado do grafo até este nó.

In [ ]:
# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# no-op node that should be interrupted on
def human_feedback(state: MessagesState):
    pass

# Assistant node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_node("human_feedback", human_feedback)

# Define edges: these determine the control flow
builder.add_edge(START, "human_feedback")
builder.add_edge("human_feedback", "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "human_feedback")

memory = MemorySaver()
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

Iremos receber feedback do usuário.

Usamos `.update_state` para atualizar o estado do grafo com a resposta humana recebida, como antes.

Usamos o parâmetro `as_node="human_feedback"` para aplicar essa atualização de estado ao nó especificado, `human_feedback`.

O parâmetro as_node é uma funcionalidade avançada do LangGraph que permite que você simule a saída de um nó específico ao atualizar o estado de uma thread manualmente (usando graph.update_state).

Em essência, ele diz ao LangGraph: "Ignore a execução do nó, mas trate esta atualização de estado como se tivesse vindo deste nó."

### Funcionamento do as_node
No seu código, a linha crucial é:

~~~Python

graph.update_state(thread, {"messages": user_input}, as_node="human_feedback")
~~~

1. O Propósito da Interrupção
O seu grafo é configurado para interromper a execução antes de entrar no nó "human_feedback":

~~~Python

graph = builder.compile(interrupt_before=["human_feedback"], ...)
~~~

Quando o grafo é executado pela primeira vez, ele para no ponto onde o nó "human_feedback" seria chamado. O LangGraph está esperando a entrada externa (o feedback humano) para continuar o fluxo.

2. A Simulação com update_state
Em vez de deixar o nó "human_feedback" executar sua função (que é uma função no-op, ou seja, não faz nada definido como: def human_feedback(state: MessagesState): pass), você usa graph.update_state para injetar dados e simular a conclusão do nó.

{"messages": user_input}: Este é o dicionário de atualização de estado que você deseja aplicar. Ele adiciona o input do usuário ao histórico de mensagens.

as_node="human_feedback": Este parâmetro tem dois efeitos principais:

Define o Ponto de Partida: Diz ao LangGraph que o estado atual deve ser atualizado e que a execução deve ser retomada a partir do ponto que se segue ao nó "human_feedback" (que, neste caso, é o nó "assistant").

Define a Fonte da Mudança: O LangGraph registra que a mudança no estado ({"messages": user_input}) foi causada pelo nó "human_feedback", mesmo que a função real human_feedback não tenha sido executada.

In [ ]:
# Input
initial_input = {"messages": "Multiply 2 and 3"}

# Thread
thread = {"configurable": {"thread_id": "5"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()
    
# Get user input
user_input = input("Tell me how you want to update the state: ")

# We now update the state as if we are the human_feedback node
graph.update_state(thread, {"messages": user_input}, as_node="human_feedback")

# Continue the graph execution
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
# Continue the graph execution
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()